In [1]:
# import all the required packages
import sys
import os
# change according to the status of GPU
# this command must before import pytorch
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,4,5'  # setting the GPUs

In [2]:

import pandas as pd
import numpy as np
import json, csv
import torch
import matplotlib.pyplot as plt
import datasets
from datasets import load_dataset, load_metric
from transformers import BertTokenizer, AutoModel

from transformers import AutoModel, BertForSequenceClassification, BertConfig
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback, IntervalStrategy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score,accuracy_score
from torch.utils.data import TensorDataset, DataLoader, random_split, RandomSampler



from sklearn.metrics import balanced_accuracy_score,f1_score,recall_score,precision_score
# metrics for multi-label classification
from sklearn.metrics import hamming_loss

/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# check the running environment
print(sys.executable)
print(torch.cuda.is_available())
torch.cuda.device_count()

/home/user/jinqli/anaconda3/envs/new_env/bin/python
True


4

## Read the prepared German data

In [4]:
de_path = '/srv/scratch2/jinq/taskontology/task_to_GWA_IWA_DWA_DE.csv'

data_de = pd.read_csv(de_path, index_col=0)
print('numbers of records in tasks to DWAs(de):', data_de.shape[0])
print('unique types of GWA', data_de['GWA Title'].nunique())
print('unique types of IWA', data_de['IWA Title'].nunique())
print('unique types of DWA', data_de['DWA Title'].nunique())

numbers of records in tasks to DWAs(de): 23543
unique types of GWA 37
unique types of IWA 332
unique types of DWA 2085


## Tokenizer

In [5]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", use_fast=True) 


## Encoding

In [24]:
def encode_data(data_path, tokenizer, y_level):
    # load the data
    dataset = load_dataset('csv', data_files=data_path,split='train')
    print('Size of the dataset: ',len(dataset))

    # encoding tasks
    encoded_data = [tokenizer(item['Task_de'], 
                              return_tensors="pt", padding='max_length', truncation=True, 
                              max_length=128, is_split_into_words=True) for item in dataset]

    # encoding labels
    y_encoded = LabelEncoder().fit_transform(dataset[y_level])
    # print('Encoded labels: ', y_encoded)

    # Zipping the tasks and the labels(GWA title) together again
    for enc_item, item in zip(encoded_data, y_encoded):
        enc_item['labels'] = torch.tensor(item)

    for item in encoded_data:
        for key in item:
            item[key] = torch.squeeze(item[key])
    
    return encoded_data


def split_data(encoded_data):
    # split into train, validation and test      
    train_set, test_val_set = train_test_split(encoded_data, test_size=0.2)
    test_set, val_set = train_test_split(test_val_set, test_size=0.5)

    print('length of the training set: ', len(train_set))
    print('length of the test set: ',len(test_set))
    print('length of the val set: ',len(val_set))
    
    # details in the dataset
    #for key, val in test_set[3].items():
    #    print(f'key: {key}, dimensions: {val.size()}')
    
    return train_set,val_set,test_set


In [25]:
encoded_data = encode_data(de_path, tokenizer, 'GWA Title')
# print('An example of zipped task and label: \n', encoded_data.__getitem__(10)) 
train_set,val_set,test_set = split_data(encoded_data)

Using custom data configuration default-2bd0bdea745cdb59
Found cached dataset csv (/home/user/jinqli/.cache/huggingface/datasets/csv/default-2bd0bdea745cdb59/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Size of the dataset:  23543
length of the training set:  18834
length of the test set:  2354
length of the val set:  2355


## Evaluation Metrics

In [8]:
# metrics for evaluation

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    balanced_accuracy = balanced_accuracy_score(labels, preds)
    f1 = f1_score(labels, preds,average='micro')
    recall = recall_score(labels, preds,average='micro')
    precision = precision_score(labels, preds,average='micro')
    hamming_loss = hamming_loss(labels, preds,average='micro')

    return {
      'accuracy': acc,
      'balanced_accuracy': balanced_accuracy,
      'f1_score': f1,
      'recall': recall,
      'precision': precision,
      'hamming_loss': hamming_loss,
    }

## Training models

### Training Parameters

In [9]:
# hyperparameters
gwa_labels=37
iwa_labels=332
dwa_labels=2085

hidden_dropout_prob = 0.3
learning_rate = 1e-5
weight_decay = 1e-2
epochs = 10
batch_size = 32

### gbert-base model

In [10]:
# load the model

german_model = "deepset/gbert-base" 
Bertmodel =  BertForSequenceClassification.from_pretrained(german_model,num_labels=gwa_labels)

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [11]:
training_args = TrainingArguments(
    num_train_epochs=epochs,
    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
    eval_steps = 100, # Evaluation and Save happens every 100 steps
    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir='./gbert_results',
    logging_dir='./gbert_logs',
    metric_for_best_model = 'hamming_loss',
    load_best_model_at_end=True
    
)

trainer_gbert = Trainer(
    model=Bertmodel,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

In [12]:
trainer_gbert.train()

***** Running training *****
  Num examples = 18834
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 1480
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Loss,Accuracy,Balanced Accuracy,F1 Score,Recall,Precision,Runtime,Samples Per Second,Steps Per Second
100,No log,0.191083,3.036555,0.191083,0.060654,0.191083,0.191083,0.191083,6.456400,364.753000,2.943000
200,No log,0.301486,2.625865,0.301486,0.136418,0.301486,0.301486,0.301486,6.458500,364.634000,2.942000
300,No log,0.392781,2.369962,0.392781,0.211314,0.392781,0.392781,0.392781,6.611900,356.177000,2.874000
400,No log,0.417834,2.232846,0.417834,0.233082,0.417834,0.417834,0.417834,6.476000,363.648000,2.934000
500,2.610700,0.451380,2.107863,0.451380,0.267190,0.451380,0.451380,0.451380,6.448000,365.230000,2.947000
600,2.610700,0.467516,2.048754,0.467516,0.285829,0.467516,0.467516,0.467516,6.604500,356.578000,2.877000
700,2.610700,0.481529,1.987307,0.481529,0.299866,0.481529,0.481529,0.481529,6.604000,356.604000,2.877000
800,2.610700,0.484926,1.961134,0.484926,0.300642,0.484926,0.484926,0.484926,6.458300,364.648000,2.942000
900,2.610700,0.491295,1.948031,0.491295,0.314623,0.491295,0.491295,0.491295,6.460700,364.510000,2.941000
1000,1.819900,0.505308,1.908834,0.505308,0.325425,0.505308,0.505308,0.505308,6.558500,359.076000,2.897000


***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tens

TrainOutput(global_step=1480, training_loss=1.996760724041913, metrics={'train_runtime': 1298.9833, 'train_samples_per_second': 144.99, 'train_steps_per_second': 1.139, 'total_flos': 1.23924771650304e+16, 'train_loss': 1.996760724041913, 'epoch': 10.0})

### job modetrainer_multibert

In [20]:
job_model = "agne/jobGBERT"
Jobmodel = BertForSequenceClassification.from_pretrained(job_model,num_labels=gwa_labels)

loading configuration file https://huggingface.co/agne/jobGBERT/resolve/main/config.json from cache at /home/user/jinqli/.cache/huggingface/transformers/d2d46e69040501cda16aa4d54a8a989fe5119ab3efd9364107329b4473dc8cbc.6efb02902914f07db4531e133a3c97080eeae52c7268a17915c9327e16cfaf1c
Model config BertConfig {
  "_name_or_path": "agne/jobGBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LA

In [21]:
training_args = TrainingArguments(
    num_train_epochs=epochs,
    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
    eval_steps = 100, # Evaluation and Save happens every 100 steps
    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir='./job_results',
    logging_dir='./job_logs',
    metric_for_best_model = 'f1_score',
    load_best_model_at_end=True
    
)

trainer_job = Trainer(
    model=Jobmodel,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:
trainer_job.train()

***** Running training *****
  Num examples = 18834
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 1480
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Loss,Accuracy,Balanced Accuracy,F1 Score,Recall,Precision,Runtime,Samples Per Second,Steps Per Second
100,No log,0.205520,2.983229,0.205520,0.066974,0.205520,0.205520,0.205520,6.454300,364.871000,2.944000
200,No log,0.308705,2.665582,0.308705,0.124907,0.308705,0.308705,0.308705,6.450500,365.086000,2.945000
300,No log,0.383439,2.378019,0.383439,0.183969,0.383439,0.383439,0.383439,6.673400,352.892000,2.847000
400,No log,0.424204,2.226835,0.424204,0.224458,0.424204,0.424204,0.424204,6.445100,365.394000,2.948000
500,2.617300,0.452654,2.125789,0.452654,0.255776,0.452654,0.452654,0.452654,6.654700,353.885000,2.855000
600,2.617300,0.461996,2.041583,0.461996,0.260881,0.461996,0.461996,0.461996,6.444300,365.438000,2.948000
700,2.617300,0.464968,2.002359,0.464968,0.271326,0.464968,0.464968,0.464968,6.667800,353.189000,2.850000
800,2.617300,0.482803,1.956388,0.482803,0.284735,0.482803,0.482803,0.482803,6.512300,361.622000,2.918000
900,2.617300,0.497665,1.914729,0.497665,0.296502,0.497665,0.497665,0.497665,6.457500,364.695000,2.942000
1000,1.841900,0.488747,1.899420,0.488747,0.294387,0.488747,0.488747,0.488747,6.599600,356.838000,2.879000


***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tens

TrainOutput(global_step=1480, training_loss=2.012536786053632, metrics={'train_runtime': 1284.6465, 'train_samples_per_second': 146.608, 'train_steps_per_second': 1.152, 'total_flos': 1.23924771650304e+16, 'train_loss': 2.012536786053632, 'epoch': 10.0})

### Multilingual model

In [15]:
from transformers import BertTokenizer, BertModel

In [16]:

multi_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

multilingual_model = "bert-base-multilingual-cased" 
Multi_Bertmodel =  BertForSequenceClassification.from_pretrained(multilingual_model, num_labels=gwa_labels)



loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at /home/user/jinqli/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /home/user/jinqli/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /home/user/jinqli/.cache/huggingface/transformers/46880f3b0081fda494a4e15b0578769

In [26]:
encoded_data = encode_data(de_path, multi_tokenizer, 'GWA Title')
train_set,val_set,test_set = split_data(encoded_data)

Using custom data configuration default-2bd0bdea745cdb59
Found cached dataset csv (/home/user/jinqli/.cache/huggingface/datasets/csv/default-2bd0bdea745cdb59/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Size of the dataset:  23543
length of the training set:  18834
length of the test set:  2354
length of the val set:  2355


In [28]:
training_args = TrainingArguments(
    num_train_epochs=epochs,
    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
    eval_steps = 100, # Evaluation and Save happens every 100 steps
    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir='./multibert_results',
    logging_dir='./multibert_logs',
    metric_for_best_model = 'f1_score',
    load_best_model_at_end=True
    
)

trainer_multibert = Trainer(
    model=Multi_Bertmodel,
    tokenizer=multi_tokenizer,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
trainer_multibert.train()

***** Running training *****
  Num examples = 18834
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 1480
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Loss,Accuracy,Balanced Accuracy,F1 Score,Recall,Precision,Runtime,Samples Per Second,Steps Per Second
100,No log,0.259023,2.714472,0.259023,0.088463,0.259023,0.259023,0.259023,7.873000,299.124000,2.413000
200,No log,0.427601,2.196005,0.427601,0.216565,0.427601,0.427601,0.427601,7.863600,299.483000,2.416000
300,No log,0.464119,1.986812,0.464119,0.257020,0.464119,0.464119,0.464119,8.154400,288.803000,2.330000
400,No log,0.501062,1.857215,0.501062,0.304660,0.501062,0.501062,0.501062,7.867000,299.353000,2.415000
500,2.211800,0.510403,1.803832,0.510403,0.331783,0.510403,0.510403,0.510403,7.862100,299.539000,2.417000
600,2.211800,0.526964,1.749402,0.526964,0.364005,0.526964,0.526964,0.526964,7.868400,299.299000,2.415000
700,2.211800,0.532059,1.734157,0.532059,0.365533,0.532059,0.532059,0.532059,7.848200,300.068000,2.421000
800,2.211800,0.524841,1.729213,0.524841,0.382370,0.524841,0.524841,0.524841,8.090700,291.076000,2.348000
900,2.211800,0.527389,1.736151,0.527389,0.387682,0.527389,0.527389,0.527389,7.850500,299.980000,2.420000
1000,1.368800,0.528662,1.729934,0.528662,0.393352,0.528662,0.528662,0.528662,7.856900,299.737000,2.418000


***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tens

TrainOutput(global_step=1480, training_loss=1.570254846521326, metrics={'train_runtime': 1516.2553, 'train_samples_per_second': 124.214, 'train_steps_per_second': 0.976, 'total_flos': 1.23924771650304e+16, 'train_loss': 1.570254846521326, 'epoch': 10.0})

In [34]:
best_ckpt_path = trainer_multibert.state.best_model_checkpoint
trainer_multibert.save_model(best_ckpt_path)

Saving model checkpoint to ./multibert_results/checkpoint-1000
Configuration saved in ./multibert_results/checkpoint-1000/config.json
Model weights saved in ./multibert_results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./multibert_results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./multibert_results/checkpoint-1000/special_tokens_map.json


### Evaluate the models performance

In [23]:
## Comparing these two models
metrics_gbert=trainer_gbert.evaluate()
print(metrics_gbert)


***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128


{'eval_loss': 1.8680411577224731, 'eval_accuracy': 0.5040339702760085, 'eval_balanced_accuracy': 0.3215903417617457, 'eval_f1_score': 0.5040339702760085, 'eval_recall': 0.5040339702760085, 'eval_precision': 0.5040339702760085, 'eval_hamming_loss': 0.5040339702760085, 'eval_runtime': 6.2722, 'eval_samples_per_second': 375.466, 'eval_steps_per_second': 3.029, 'epoch': 8.78}


In [24]:

metrics_job=trainer_job.evaluate()
print(metrics_job)

***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.8994202613830566, 'eval_accuracy': 0.48874734607218684, 'eval_balanced_accuracy': 0.29438654919459367, 'eval_f1_score': 0.48874734607218684, 'eval_recall': 0.48874734607218684, 'eval_precision': 0.48874734607218684, 'eval_hamming_loss': 0.48874734607218684, 'eval_runtime': 6.2803, 'eval_samples_per_second': 374.979, 'eval_steps_per_second': 3.025, 'epoch': 10.0}


In [31]:
metrics_multibert=trainer_multibert.evaluate()
print(metrics_multibert)

***** Running Evaluation *****
  Num examples = 2355
  Batch size = 128
/home/user/jinqli/anaconda3/envs/new_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.7299339771270752, 'eval_accuracy': 0.5286624203821656, 'eval_balanced_accuracy': 0.3933516082476829, 'eval_f1_score': 0.5286624203821656, 'eval_recall': 0.5286624203821656, 'eval_precision': 0.5286624203821656, 'eval_hamming_loss': 0.5286624203821656, 'eval_runtime': 7.4651, 'eval_samples_per_second': 315.469, 'eval_steps_per_second': 2.545, 'epoch': 10.0}
